In [2]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from imblearn.over_sampling import SMOTE, ADASYN
sm = SMOTE(random_state = 2)
ada = ADASYN(random_state = 42, sampling_strategy='minority')
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Using TensorFlow backend.


In [3]:
def g11(val): 
    """
    Records whether or not a number is greater than 11. 
    """
    if val <= 11: 
        return 0
    else: 
        return 1

In [4]:
df = pd.read_csv('fservice.csv')
df['Eleven'] = df['ElapsedDays'].apply(g11, 0)

values = {'MobileOS': "NotMobile", 'CreatedByUserOrganization': 'NotSpecified'}

df = df.fillna(value=values)

/Users/varunbopardikar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df['Just Date'] = df['Just Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [6]:
df['day_of_week'] = df['Just Date'].dt.day_name()

In [7]:
df['Month'] = df['Just Date'].dt.month_name()

In [11]:
#Columns 

#c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month'] #BEST

c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month']


n = ['Latitude', 'Longitude']

In [12]:
dfm = df[df['Eleven'] == 1]

In [13]:
dfl = df[df['Eleven'] == 0].sample(dfm.shape[0])

In [14]:
dfn = dfl.append(dfm)
dfn = dfn.filter(items = c + n  + ['Eleven'])
dfn = dfn.dropna()

In [16]:
#Separate data into explanatory and response variables
XCAT = dfn.filter(items = c).values
XNUM = dfn.filter(items = n).values

y = np.array(dfn['Eleven'].values, dtype=np.float)

#Encode cateogrical data and merge with numerical data
labelencoder_X = LabelEncoder()
for num in range(len(c)):
    XCAT[:, num] = labelencoder_X.fit_transform(XCAT[:, num])

onehotencoder = OneHotEncoder()
XCAT = onehotencoder.fit_transform(XCAT).toarray()

X = np.concatenate((XCAT, XNUM), axis=1)

In [23]:
dfn['RequestType'].unique()

array(['Graffiti Removal', 'Bulky Items', 'Metal/Household Appliances',
       'Dead Animal Removal', 'Multiple Streetlight Issue',
       'Illegal Dumping Pickup', 'Electronic Waste',
       'Homeless Encampment', 'Other', 'Single Streetlight Issue',
       'Report Water Waste', 'Feedback'], dtype=object)

In [434]:
#Split data 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

rf = RandomForestClassifier(n_estimators = 50, max_depth = 20)  #Current best
et = ExtraTreesClassifier(n_estimators=10, max_depth=20, min_samples_split=2, random_state=0)
ab = AdaBoostClassifier(n_estimators=50)
gb = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=20, random_state=0)


classifier = rf
classifier.fit(X_train, y_train)


##TEST SET##
y_tpred = classifier.predict(X_test)

#Print Test results
print("Test Set Results")
print(metrics.classification_report(y_test, y_tpred, digits=3))

Test Set Results
              precision    recall  f1-score   support

         0.0      0.842     0.834     0.838     31526
         1.0      0.845     0.852     0.848     33331

    accuracy                          0.843     64857
   macro avg      0.843     0.843     0.843     64857
weighted avg      0.843     0.843     0.843     64857



In [317]:
df.columns

Index(['APC', 'ActionTaken', 'Address', 'AddressVerified', 'Anonymous',
       'ApproximateAddress', 'AssignTo', 'CD', 'CDMember', 'ClosedDate',
       'CreatedByUserOrganization', 'CreatedDate', 'Direction', 'HouseNumber',
       'Latitude', 'Location', 'Longitude', 'MobileOS', 'NC', 'NCName',
       'Owner', 'PolicePrecinct', 'RequestSource', 'RequestType', 'SRNumber',
       'ServiceDate', 'Status', 'StreetName', 'Suffix', 'TBMColumn', 'TBMPage',
       'TBMRow', 'UpdatedDate', 'ZipCode', 'ElapsedTime', 'ElapsedDays',
       'Just Date', 'Eleven', 'day_of_week', 'Month'],
      dtype='object')

In [25]:
dfa = dfl.append(dfm)

In [26]:
dfa[dfa['CreatedByUserOrganization'] == "ITA"]['Eleven'].value_counts()

1    10658
0     5596
Name: Eleven, dtype: int64

In [27]:
dfa['CreatedByUserOrganization'].unique()

array(['NotSpecified', 'BOS', 'Self Service', 'Proactive Insert', 'OCB',
       'ITA', 'Self Service_SAN', "Council's Office", 'BSS',
       'BOS Franchise', 'BSL', 'LAAS'], dtype=object)

In [28]:
c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month']